In [8]:
url = "https://www.glassdoor.com/Reviews/North-South-University-Reviews-E465930.htm"

In [9]:
import re

pattern = r"E(\d+)"
match = re.search(pattern, url)

if match:
    employerId = match.group(1)
else:
    employerId = None  # Or you can set a default value if needed

print(f"Employer ID: {employerId}")

Employer ID: 465930


In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

EMAIL = "***"
PASSWORD = "***"

# Declare global variables
gd_csrf_token = None
cookie = None

async def login_and_extract():
    global gd_csrf_token, cookie

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        async def handle_request(request):
            global gd_csrf_token
            if 'gd-csrf-token' in request.headers:
                gd_csrf_token = request.headers['gd-csrf-token']

        page.on("request", handle_request)

        # Step 1: Go to login page
        await page.goto("https://www.glassdoor.com/profile/login_input.htm", wait_until="domcontentloaded", timeout=60000)

        # Step 2: Email input
        await page.wait_for_selector('input#inlineUserEmail', timeout=15000)
        await page.fill('input#inlineUserEmail', EMAIL)
        await page.click('button[data-test="email-form-button"]')
        await page.wait_for_timeout(2000)

        # Step 3: Password input
        await page.wait_for_selector('input#inlineUserPassword', timeout=15000)
        await page.fill('input#inlineUserPassword', PASSWORD)
        await page.click('form[name="authEmailForm"] button[type="submit"]')

        await page.wait_for_timeout(5000)

        # Step 4: Extract cookies
        cookies = await context.cookies()
        cookie = "; ".join([f"{c['name']}={c['value']}" for c in cookies])

        current_url = page.url
        page_content = await page.content()

        print("\n✅ Extraction Result:")
        print(f"Current URL: {current_url}")
        print(f"gd-csrf-token: {gd_csrf_token}")
        print(f"Cookies: {cookie}")

        if "Session Timeout" in page_content:
            print("⚠️ Page says: Session Timeout")
        else:
            print("✅ Page did not mention Session Timeout explicitly.")

        await browser.close()

await login_and_extract()



✅ Extraction Result:
Current URL: https://www.glassdoor.com/profile/login_input.htm
gd-csrf-token: KDvgFRAWsNYYpEty3b6WWw:g-abkMtkMw_SvvMEOejQzyXieyRSwjVFtZKRynrvTpATIwJ-sx7t0tuv1hao6_m0WvflS06BPMxDNHj_ZThORw:YQ0mBqRA-ukJLWyNULQrABgJbZiYhXErZqbKitnIUIA
Cookies: gdsid=1744956894831:1744956894831:286069C9CA9572498DD9B314A31FAD0A; gdId=5d3d2a55-15df-47ed-ba63-168fb85e39d8; asst=1744956894.0; bs=d8oKHOSvdmiFWsLeZinG8g:10ylIO3PjyW0Gcr_Kx8uZIGLIRJXYWkDO39Y2yDIyxSCr8GKM2y1_n3wN7foFThDtuq-qX0NlG6t_O64czOddd1Z9RzkBTUZp_r-ZeDU98c:BRqcFqC-2jPaRRsM5lgq_pCs6eSAaMA9rBbdmm6byPQ; __cf_bm=s.T00R8EPWU0XP4gOKUbq3WEdZCZRYEcnkM5zVg.Faw-1744956895-1.0.1.1-SqgCf5wFz_7t6EXTahy8AaeGnVuYszWAYs9_7CKG6a23bSXt.YzUx2OK33_Yz6YTOBYuA3hiPNy7wQKf7c4briA31x0Se28JGjXYTgYU2oldwMAIKyU641rtMRqNdY_n; _cfuvid=x3zlZPXTpGfIBrvx7PtHOEdyPFGZUrmac17xnJQPVA4-1744956895162-0.0.1.1-604800000; rl_user_id=RudderEncrypt%3AU2FsdGVkX18UI6F%2F7KgoAJ0MaphGfED9CEDx1bzRm%2Fs%3D; rl_trait=RudderEncrypt%3AU2FsdGVkX1%2Ba1cV7hwYuCFxSmja4Zv7hXhx5

In [11]:
import requests
import json

url = "https://www.glassdoor.com/graph"

payload = json.dumps([
  {
    "operationName": "RecordPageView",
    "variables": {
      "employerId": employerId, ######Changinggggg
      "pageIdent": "INFOSITE_REVIEWS"
    },
    "query": "mutation RecordPageView($employerId: String!, $pageIdent: String!) {\n  recordPageView(\n    pageIdent: $pageIdent\n    metaData: {key: \"employerId\", value: $employerId}\n  ) {\n    totalCount\n    __typename\n  }\n}\n"
  },
  {
    "operationName": "GetEmployerReviews",
    "variables": {
      "applyDefaultCriteria": True,
      "employerId": int(employerId),   ######Changinggggg
      "employmentStatuses": [],
      "goc": None,
      "jobTitle": None,
      "location": {
        "countryId": None,
        "stateId": None,
        "metroId": None,
        "cityId": None
      },
      "onlyCurrentEmployees": False,
      "overallRating": None,
      "page": 1,
      "preferredTldId": 0,
      "reviewCategories": [],
      "sort": "RELEVANCE",
      "textSearch": "",
      "worldwideFilter": False,
      "language": "eng",
      #"dynamicProfileId": 8370167,   ######Changinggggg
      "useRowProfileTldForRatings": False,
      "enableKeywordSearch": False
    },
    "query": "query GetEmployerReviews($applyDefaultCriteria: Boolean, $dynamicProfileId: Int, $employerId: Int!, $employmentStatuses: [EmploymentStatusEnum], $enableKeywordSearch: Boolean!, $goc: GOCIdent, $isRowProfileEnabled: Boolean, $jobTitle: JobTitleIdent, $language: String, $languageOverrides: [String], $location: LocationIdent, $onlyCurrentEmployees: Boolean, $overallRating: FiveStarRatingEnum, $page: Int!, $preferredTldId: Int, $reviewCategories: [ReviewCategoriesEnum], $sort: ReviewsSortOrderEnum, $textSearch: String, $useRowProfileTldForRatings: Boolean, $worldwideFilter: Boolean) {\n  employerReviews: employerReviewsRG(\n    employerReviewsInput: {applyDefaultCriteria: $applyDefaultCriteria, dynamicProfileId: $dynamicProfileId, employer: {id: $employerId}, employmentStatuses: $employmentStatuses, onlyCurrentEmployees: $onlyCurrentEmployees, goc: $goc, isRowProfileEnabled: $isRowProfileEnabled, jobTitle: $jobTitle, language: $language, languageOverrides: $languageOverrides, location: $location, overallRating: $overallRating, page: {num: $page, size: 10}, preferredTldId: $preferredTldId, reviewCategories: $reviewCategories, sort: $sort, textSearch: $textSearch, useRowProfileTldForRatings: $useRowProfileTldForRatings, worldwideFilter: $worldwideFilter}\n  ) {\n    allReviewsCount\n    currentPage\n    filteredReviewsCount\n    lastReviewDateTime\n    numberOfPages\n    queryJobTitle {\n      id\n      text\n      mgocId\n      __typename\n    }\n    queryLocation {\n      id\n      longName\n      shortName\n      type\n      __typename\n    }\n    ratedReviewsCount\n    ratings {\n      businessOutlookRating\n      careerOpportunitiesRating\n      ceoRating\n      compensationAndBenefitsRating\n      cultureAndValuesRating\n      diversityAndInclusionRating\n      overallRating\n      ratedCeo {\n        id\n        largePhoto: photoUrl(size: LARGE)\n        name\n        regularPhoto: photoUrl(size: REGULAR)\n        title\n        __typename\n      }\n      recommendToFriendRating\n      reviewCount\n      seniorManagementRating\n      workLifeBalanceRating\n      __typename\n    }\n    reviews {\n      advice\n      adviceOriginal\n      cons\n      consOriginal\n      countHelpful\n      countNotHelpful\n      employer {\n        id\n        largeLogoUrl: squareLogoUrl(size: LARGE)\n        regularLogoUrl: squareLogoUrl(size: REGULAR)\n        shortName\n        __typename\n      }\n      employerResponses {\n        id\n        countHelpful\n        countNotHelpful\n        languageId\n        originalLanguageId\n        response\n        responseDateTime(format: ISO)\n        responseOriginal\n        translationMethod\n        __typename\n      }\n      employmentStatus\n      flaggingDisabled\n      featured\n      isCurrentJob\n      jobTitle {\n        id\n        text\n        __typename\n      }\n      languageId\n      lengthOfEmployment\n      location {\n        id\n        type\n        name\n        __typename\n      }\n      originalLanguageId\n      pros\n      prosOriginal\n      ratingBusinessOutlook\n      ratingCareerOpportunities\n      ratingCeo\n      ratingCompensationAndBenefits\n      ratingCultureAndValues\n      ratingDiversityAndInclusion\n      ratingOverall\n      ratingRecommendToFriend\n      ratingSeniorLeadership\n      ratingWorkLifeBalance\n      reviewDateTime\n      reviewId\n      relatedStructures {\n        companyStructureId\n        companyStructureName\n        __typename\n      }\n      summary\n      summaryOriginal\n      textSearchHighlightPhrases @include(if: $enableKeywordSearch) {\n        field\n        phrases {\n          length\n          position: pos\n          __typename\n        }\n        __typename\n      }\n      translationMethod\n      __typename\n    }\n    ratingCountDistribution {\n      overall {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      cultureAndValues {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      careerOpportunities {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      workLifeBalance {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      seniorManagement {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      compensationAndBenefits {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      diversityAndInclusion {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      recommendToFriend {\n        WONT_RECOMMEND\n        RECOMMEND\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'accept': '*/*',
  'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8,bn;q=0.7',
  'apollographql-client-name': 'ei-reviews-next',
  'apollographql-client-version': '1.93.0',
  'content-type': 'application/json',
  'gd-csrf-token': gd_csrf_token,
  'origin': 'https://www.glassdoor.com',
  'priority': 'u=1, i',
  'referer': 'https://www.glassdoor.com/',
  'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
  'sec-ch-ua-arch': '"arm"',
  'sec-ch-ua-bitness': '"64"',
  'sec-ch-ua-full-version': '"135.0.7049.85"',
  'sec-ch-ua-full-version-list': '"Google Chrome";v="135.0.7049.85", "Not-A.Brand";v="8.0.0.0", "Chromium";v="135.0.7049.85"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-model': '""',
  'sec-ch-ua-platform': '"macOS"',
  'sec-ch-ua-platform-version': '"15.4.0"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
  'Cookie': cookie
}

response = requests.request("POST", url, headers=headers, data=payload)
response_data = response.json()
number_of_pages = response_data[1]['data']['employerReviews']['numberOfPages']
print(f"Total Review Page: {number_of_pages}")


Total Review Page: 20


In [12]:
import requests
import json
import os

# Check if the 'resp' folder exists, and if it does, find the next available folder name
folder_name = "resp"
counter = 1

while os.path.exists(folder_name):
    folder_name = f"resp{counter}"
    counter += 1

# Create the folder to store the responses
os.makedirs(folder_name)

url = "https://www.glassdoor.com/graph"

for page in range(1, number_of_pages + 1):  # Loop from 1 to number_of_pages
    payload = json.dumps([
      {
        "operationName": "RecordPageView",
        "variables": {
          "employerId": employerId,  # Changinggggg
          "pageIdent": "INFOSITE_REVIEWS"
        },
        "query": "mutation RecordPageView($employerId: String!, $pageIdent: String!) {\n  recordPageView(\n    pageIdent: $pageIdent\n    metaData: {key: \"employerId\", value: $employerId}\n  ) {\n    totalCount\n    __typename\n  }\n}\n"
      },
      {
        "operationName": "GetEmployerReviews",
        "variables": {
          "applyDefaultCriteria": True,
          "employerId": int(employerId),  # Changinggggg
          "employmentStatuses": [],
          "goc": None,
          "jobTitle": None,
          "location": {
            "countryId": None,
            "stateId": None,
            "metroId": None,
            "cityId": None
          },
          "onlyCurrentEmployees": False,
          "overallRating": None,
          "page": int(page),  # Replacing the page dynamically
          "preferredTldId": 0,
          "reviewCategories": [],
          "sort": "RELEVANCE",
          "textSearch": "",
          "worldwideFilter": False,
          "language": "eng",
          "useRowProfileTldForRatings": False,
          "enableKeywordSearch": False
        },
        "query": "query GetEmployerReviews($applyDefaultCriteria: Boolean, $dynamicProfileId: Int, $employerId: Int!, $employmentStatuses: [EmploymentStatusEnum], $enableKeywordSearch: Boolean!, $goc: GOCIdent, $isRowProfileEnabled: Boolean, $jobTitle: JobTitleIdent, $language: String, $languageOverrides: [String], $location: LocationIdent, $onlyCurrentEmployees: Boolean, $overallRating: FiveStarRatingEnum, $page: Int!, $preferredTldId: Int, $reviewCategories: [ReviewCategoriesEnum], $sort: ReviewsSortOrderEnum, $textSearch: String, $useRowProfileTldForRatings: Boolean, $worldwideFilter: Boolean) {\n  employerReviews: employerReviewsRG(\n    employerReviewsInput: {applyDefaultCriteria: $applyDefaultCriteria, dynamicProfileId: $dynamicProfileId, employer: {id: $employerId}, employmentStatuses: $employmentStatuses, onlyCurrentEmployees: $onlyCurrentEmployees, goc: $goc, isRowProfileEnabled: $isRowProfileEnabled, jobTitle: $jobTitle, language: $language, languageOverrides: $languageOverrides, location: $location, overallRating: $overallRating, page: {num: $page, size: 10}, preferredTldId: $preferredTldId, reviewCategories: $reviewCategories, sort: $sort, textSearch: $textSearch, useRowProfileTldForRatings: $useRowProfileTldForRatings, worldwideFilter: $worldwideFilter}\n  ) {\n    allReviewsCount\n    currentPage\n    filteredReviewsCount\n    lastReviewDateTime\n    numberOfPages\n    queryJobTitle {\n      id\n      text\n      mgocId\n      __typename\n    }\n    queryLocation {\n      id\n      longName\n      shortName\n      type\n      __typename\n    }\n    ratedReviewsCount\n    ratings {\n      businessOutlookRating\n      careerOpportunitiesRating\n      ceoRating\n      compensationAndBenefitsRating\n      cultureAndValuesRating\n      diversityAndInclusionRating\n      overallRating\n      ratedCeo {\n        id\n        largePhoto: photoUrl(size: LARGE)\n        name\n        regularPhoto: photoUrl(size: REGULAR)\n        title\n        __typename\n      }\n      recommendToFriendRating\n      reviewCount\n      seniorManagementRating\n      workLifeBalanceRating\n      __typename\n    }\n    reviews {\n      advice\n      adviceOriginal\n      cons\n      consOriginal\n      countHelpful\n      countNotHelpful\n      employer {\n        id\n        largeLogoUrl: squareLogoUrl(size: LARGE)\n        regularLogoUrl: squareLogoUrl(size: REGULAR)\n        shortName\n        __typename\n      }\n      employerResponses {\n        id\n        countHelpful\n        countNotHelpful\n        languageId\n        originalLanguageId\n        response\n        responseDateTime(format: ISO)\n        responseOriginal\n        translationMethod\n        __typename\n      }\n      employmentStatus\n      flaggingDisabled\n      featured\n      isCurrentJob\n      jobTitle {\n        id\n        text\n        __typename\n      }\n      languageId\n      lengthOfEmployment\n      location {\n        id\n        type\n        name\n        __typename\n      }\n      originalLanguageId\n      pros\n      prosOriginal\n      ratingBusinessOutlook\n      ratingCareerOpportunities\n      ratingCeo\n      ratingCompensationAndBenefits\n      ratingCultureAndValues\n      ratingDiversityAndInclusion\n      ratingOverall\n      ratingRecommendToFriend\n      ratingSeniorLeadership\n      ratingWorkLifeBalance\n      reviewDateTime\n      reviewId\n      relatedStructures {\n        companyStructureId\n        companyStructureName\n        __typename\n      }\n      summary\n      summaryOriginal\n      textSearchHighlightPhrases @include(if: $enableKeywordSearch) {\n        field\n        phrases {\n          length\n          position: pos\n          __typename\n        }\n        __typename\n      }\n      translationMethod\n      __typename\n    }\n    ratingCountDistribution {\n      overall {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      cultureAndValues {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      careerOpportunities {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      workLifeBalance {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      seniorManagement {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      compensationAndBenefits {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      diversityAndInclusion {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      recommendToFriend {\n        WONT_RECOMMEND\n        RECOMMEND\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
      }
    ])

    headers = {
      'accept': '*/*',
      'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8,bn;q=0.7',
      'apollographql-client-name': 'ei-reviews-next',
      'apollographql-client-version': '1.93.0',
      'content-type': 'application/json',
      'gd-csrf-token': gd_csrf_token,
      'origin': 'https://www.glassdoor.com',
      'priority': 'u=1, i',
      'referer': 'https://www.glassdoor.com/',
      'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
      'sec-ch-ua-arch': '"arm"',
      'sec-ch-ua-bitness': '"64"',
      'sec-ch-ua-full-version': '"135.0.7049.85"',
      'sec-ch-ua-full-version-list': '"Google Chrome";v="135.0.7049.85", "Not-A.Brand";v="8.0.0.0", "Chromium";v="135.0.7049.85"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-model': '""',
      'sec-ch-ua-platform': '"macOS"',
      'sec-ch-ua-platform-version': '"15.4.0"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-origin',
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
      'Cookie': cookie
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    
    # Save the response as a JSON file for each page in the new folder
    with open(f"{folder_name}/pg{page}.json", "w") as file:
        json.dump(response.json(), file, indent=4)

    print(f"Page {page} saved to {folder_name}/pg{page}.json")

Page 1 saved to resp2/pg1.json
Page 2 saved to resp2/pg2.json
Page 3 saved to resp2/pg3.json
Page 4 saved to resp2/pg4.json
Page 5 saved to resp2/pg5.json
Page 6 saved to resp2/pg6.json
Page 7 saved to resp2/pg7.json
Page 8 saved to resp2/pg8.json
Page 9 saved to resp2/pg9.json
Page 10 saved to resp2/pg10.json
Page 11 saved to resp2/pg11.json
Page 12 saved to resp2/pg12.json
Page 13 saved to resp2/pg13.json
Page 14 saved to resp2/pg14.json
Page 15 saved to resp2/pg15.json
Page 16 saved to resp2/pg16.json
Page 17 saved to resp2/pg17.json
Page 18 saved to resp2/pg18.json
Page 19 saved to resp2/pg19.json
Page 20 saved to resp2/pg20.json
